In [1]:
import numpy as np
import pyomo.environ as pyo
import pandas as pd
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.parmest import parmest

## Data

In [2]:
data = {
    'Age (years)': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 11, 12, 12, 12],
    'PCB Concentration (ppm)': [0.6, 1.6, 0.5, 1.2, 2.0, 1.3, 2.5, 2.2, 2.4, 1.2, 3.5, 4.1, 5.1, 5.7, 3.4, 9.7, 8.6, 4.0, 5.5, 10.5, 17.5, 13.4, 4.5, 30.4, 12.4, 13.4, 26.2, 7.4]
}

## Creating a pyomo model

In [11]:
def pyomo_model(data):
    # Create the concrete model
    model = pyo.ConcreteModel()

    # Add parameters and variables in the concrete model
    # Vars
    model.m = pyo.Var(initialize = 0.2)  # Slope
    model.b = pyo.Var(initialize = 0.03)  # intercept

    # Given data values
    model.age = pyo.Param(initialize = float(data["Age (years)"]))
    # model.ln_pcb = pyo.Param(initialize = np.log(float(data["PCB Concentration"])))

    # We will express the equation relating the paramaters and vars as an `Expression`
    # equation: ln(PCB) = m + b * age
    model.pcb = pyo.Expression(expr = pyo.exp(model.b + model.m * model.age))

    # fix all regressed parameters (b, m)
    model.m.fix()
    model.b.fix()
    return model


## Creating AgeFind class

In [7]:
class PCB_Conc(Experiment):
    
    # Defining the constructor
    def __init__(self, data):
        self.data = data
        self.model = None

    
    def get_labeled_model(self):
        """
        get_labeled_model is a mandatory method in both parmest and DOE. In this method, we 
        build the model, and then finalize it.
        """
        if self.model == None:
            self.create_model()
            self.label_model()
            return self.model


    # Create flixible model without data
    def create_model(self):    
        """
        Here, we will create different variables, parameters, and constraints
        """  
        self.model = pyomo_model(self.data)  
        # we don't need to create new model, since we have already created a model in pyomo_model.
        # so we are just calling that function here.
        
        return self.model

    
    def label_model(self):
        """
        The model is updated with outputs, inputs, errors and unknown parameters
        This makes the model labeled with full experiment
        """
        model = self.model

        # Experiment outputs
        model.experiment_outputs = pyo.Suffix(direction = pyo.Suffix.LOCAL)
        # passing the experiment data as a `list` of `tuple` in the experiment_outputs
        model.experiment_outputs.update([
            (model.pcb, self.data['PCB Concentration']),
            (model.age, self.data['Age (years)'])
            
        ])

        model.unknown_parameters = pyo.Suffix(direction = pyo.Suffix.LOCAL)
        model.unknown_parameters[model.m] = 0.2
        model.unknown_parameters[model.b] = 0.03

        return model          

## Creating list of experiments from the given experimental data

In [8]:
data_lc = [
    {"Age (years)": age, "PCB Concentration": PCB_Concentration} for age, PCB_Concentration in zip(data['Age (years)'], data['PCB Concentration (ppm)'])
]

exp_list = []
n_data = len(data_lc)
for i in range(n_data):
    exp_list.append(AgeFind(data_lc[i]))
    

pest = parmest.Estimator( exp_list, obj_function = "SSE")

AttributeError: 'NoneType' object has no attribute 'unknown_parameters'

In [ ]:
type(exp_list["Age (years)"])

In [9]:
type(exp_list)

list

In [10]:
data_lc

[{'Age (years)': 1, 'PCB Concentration': 0.6},
 {'Age (years)': 1, 'PCB Concentration': 1.6},
 {'Age (years)': 1, 'PCB Concentration': 0.5},
 {'Age (years)': 1, 'PCB Concentration': 1.2},
 {'Age (years)': 2, 'PCB Concentration': 2.0},
 {'Age (years)': 2, 'PCB Concentration': 1.3},
 {'Age (years)': 2, 'PCB Concentration': 2.5},
 {'Age (years)': 3, 'PCB Concentration': 2.2},
 {'Age (years)': 3, 'PCB Concentration': 2.4},
 {'Age (years)': 3, 'PCB Concentration': 1.2},
 {'Age (years)': 4, 'PCB Concentration': 3.5},
 {'Age (years)': 4, 'PCB Concentration': 4.1},
 {'Age (years)': 4, 'PCB Concentration': 5.1},
 {'Age (years)': 5, 'PCB Concentration': 5.7},
 {'Age (years)': 6, 'PCB Concentration': 3.4},
 {'Age (years)': 6, 'PCB Concentration': 9.7},
 {'Age (years)': 6, 'PCB Concentration': 8.6},
 {'Age (years)': 7, 'PCB Concentration': 4.0},
 {'Age (years)': 7, 'PCB Concentration': 5.5},
 {'Age (years)': 7, 'PCB Concentration': 10.5},
 {'Age (years)': 8, 'PCB Concentration': 17.5},
 {'Age (yea